# 1. Получение эмбеддингов 

#### Импортирование необходимых бибилиотек

In [12]:
import pandas as pd
import string
from tqdm import tqdm
import warnings
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer

en_stopwords = stopwords.words('english')

#### Загрузка данных

In [7]:
df = pd.read_csv('train.csv')
df = df.rename(columns={'Description': 'text'})
df.head()

,text,Label
0,the ultimate guide to ielts writing about the ...,1
1,by pioneer bdp180 network 3d bluray player wit...,3
2,vkcreation womens new designer silk semistiche...,2
3,portronics sound slick ii por936 wireless blue...,3
4,rebuilding india were the last four years tran...,1


#### Первичный анализ данных

In [11]:
# Подсчет пропусков (пустых строк) и дубликатов
count_missing = df['text'].isna().sum() + (df['text'] == '').sum()
count_duplicates = df.duplicated().sum()

print("Анализ данных:")
print("--------------------------")
print(f"Исходный размер выборки: {len(df)}")
print(f"Количество пустых строк (пропусков): {count_missing}")
print(f"Количество дубликатов в датафрейме: {count_duplicates}")

# Удаление дубликатов и обновление размера выборки
df = df.drop_duplicates().reset_index(drop=True)
print("--------------------------")
print(f"Итоговый размер выборки: {len(df)}")



Анализ данных:
--------------------------
Исходный размер выборки: 26330
Количество пустых строк (пропусков): 0
Количество дубликатов в датафрейме: 0
--------------------------
Итоговый размер выборки: 26330


#### Предобработка текста

In [10]:
wordnet = WordNetLemmatizer()

def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Удаление лишних пробелов
    text = " ".join(text.split())
    
    # Токенизация текста
    tokens = word_tokenize(text)
    
    # Удаление стоп-слов
    tokens = [token for token in tokens if token not in en_stopwords]
    
    # Удаление знаков препинания
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Лемматизация токенов
    lemmatized_tokens = []
    for token, tag in pos_tag(tokens):
        pos = tag[0].lower() if tag[0].lower() in ['a', 'r', 'n', 'v'] else 'n'
        lemmatized_tokens.append(wordnet.lemmatize(token, pos))
    
    # Объединение лемматизированных токенов в строку
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

df['prep_text'] = df['text'].apply(preprocess_text)
df.head()

,text,Label,prep_text
0,the ultimate guide to ielts writing about the ...,1,ultimate guide ielts write book ultimate guide...
1,by pioneer bdp180 network 3d bluray player wit...,3,pioneer bdp180 network 3d bluray player 4k vid...
2,vkcreation womens new designer silk semistiche...,2,vkcreation womens new designer silk semistiche...
3,portronics sound slick ii por936 wireless blue...,3,portronics sound slick ii por936 wireless blue...
4,rebuilding india were the last four years tran...,1,rebuild india last four year transformative en...


#### Загрузка LLM моделей

In [11]:
warnings.filterwarnings("ignore")
model_1_all_mpnet_base_v2 = SentenceTransformer("all-mpnet-base-v2")
model_2_all_MiniLM_L6_v2 = SentenceTransformer("all-MiniLM-L6-v2")
model_3_USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

#### Создание датафреймов с энбеддингами

In [13]:
def get_sentence_embedding(model_type, text, model_idx):
    if model_idx == 2:
        embeddings = model_type([text])
        return embeddings.numpy()[0] 
    else:
        embeddings = model_type.encode([text])  
        return embeddings[0]

models_list = [
    model_1_all_mpnet_base_v2,
    model_2_all_MiniLM_L6_v2,
    model_3_USE
]

tqdm.pandas()  

# Создание датафрейма с эмбеддингами исходного текста
for model_idx, model_type in enumerate(models_list):
    new_col_name = f"{model_idx + 1}_text"
    print(new_col_name)
    df[new_col_name] = df['text'].progress_apply(lambda text: get_sentence_embedding(model_type, text, model_idx))

df.to_csv('all_embeddings_text.csv')

# Создание датафрейма с эмбеддингами предобработанного текста
for model_idx, model_type in enumerate(models_list):
    new_col_name = f"{model_idx + 1}_prep_text"
    print(new_col_name)
    df[new_col_name] = df['prep_text'].progress_apply(lambda text: get_sentence_embedding(model_type, text, model_idx))

df.to_csv('all_embeddings_prep_text.csv')

1_text


100%|██████████| 26330/26330 [43:41<00:00, 10.04it/s] 


2_text


100%|██████████| 26330/26330 [09:31<00:00, 46.08it/s]


3_text


100%|██████████| 26330/26330 [00:51<00:00, 507.32it/s]
